In [3]:
import os
import time
import urllib.request
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [6]:
def download_google_images(query, limit=10):
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--log-level=3")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)

    driver = webdriver.Chrome(options=chrome_options)
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
        Object.defineProperty(navigator, 'webdriver', {get: () => undefined});
        """
    })

    wait = WebDriverWait(driver, 10)
    driver.get("https://www.google.com/imghp")

    try:
        accept = wait.until(EC.element_to_be_clickable((By.XPATH, '//button/div[normalize-space()="Tout accepter"]')))
        accept.click()
    except:
        pass

    search_box = wait.until(EC.presence_of_element_located((By.NAME, "q")))
    search_box.send_keys(query)
    search_box.send_keys(Keys.RETURN)

    time.sleep(0.5)
    thumbnails = driver.find_elements(By.CSS_SELECTOR, "img.YQ4gaf")

    if not os.path.exists("images/" + query):
        os.makedirs("images/" + query)

    count = 0
    for img in thumbnails:
        try:
            # Vérifie que ce n’est pas un logo de site
            parent_classes = img.get_attribute("class")
            if "zr758" in parent_classes:
                continue  # skip les logos

            driver.execute_script("arguments[0].scrollIntoView(true);", img)
            img.click()
            WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "img.iPVvYb"))
            )


            actual_images = driver.find_elements(By.CSS_SELECTOR, "img.iPVvYb")
            for actual_img in actual_images:
                src = actual_img.get_attribute("src")
                if src and "http" in src:
                    filename = os.path.join("images/"+query, f"{query}_{count}.jpg")
                    urllib.request.urlretrieve(src, filename)
                    print(f"[✓] {filename}")
                    count += 1
                    break
        except Exception as e:
            print("Erreur :", e)

        if count >= limit:
            break

    driver.quit()
    print("Téléchargement terminé.")

download_google_images("elephant", limit=100)

[✓] images/elephant\elephant_0.jpg
Erreur : HTTP Error 403: Forbidden
[✓] images/elephant\elephant_1.jpg
[✓] images/elephant\elephant_2.jpg
[✓] images/elephant\elephant_3.jpg
[✓] images/elephant\elephant_4.jpg
[✓] images/elephant\elephant_5.jpg
[✓] images/elephant\elephant_6.jpg
[✓] images/elephant\elephant_7.jpg
[✓] images/elephant\elephant_8.jpg
[✓] images/elephant\elephant_9.jpg
[✓] images/elephant\elephant_10.jpg
[✓] images/elephant\elephant_11.jpg
[✓] images/elephant\elephant_12.jpg
[✓] images/elephant\elephant_13.jpg
[✓] images/elephant\elephant_14.jpg
[✓] images/elephant\elephant_15.jpg
Erreur : HTTP Error 403: Forbidden
[✓] images/elephant\elephant_16.jpg
[✓] images/elephant\elephant_17.jpg
Erreur : HTTP Error 403: Forbidden
[✓] images/elephant\elephant_18.jpg
[✓] images/elephant\elephant_19.jpg
[✓] images/elephant\elephant_20.jpg
[✓] images/elephant\elephant_21.jpg
Erreur : Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff74f2e6f95+76917]
	GetHandleVerifier [0x0x7ff74f2e6ff0+7